In [3]:
import serial
import time
from IPython.display import display, clear_output
from cryptography.hazmat.primitives.ciphers.aead import AESCCM

In [2]:
SERIAL_PORT = "/dev/cu.usbmodem101"
BAUD_RATE = 115200

ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.1)
time.sleep(2)               # give the ESP time to reboot+enumerate
ser.reset_input_buffer()    # drop any boot messages

In [3]:
def send_ping_and_wait(timeout_s=5.0):
    ser.write(b"ping\n")
    deadline = time.time() + timeout_s
    while time.time() < deadline:
        raw = ser.readline()
        if not raw:
            continue
        line = raw.decode(errors="ignore").strip()
        # debug: print any other lines (log lines) you happen to see
        if line.startswith("ping:"):
            print("OK:", line)
            return
        else:
            print("  log >", line)
    raise TimeoutError("no ping reply in time")

In [4]:
def send_log_dump_and_wait(timeout_s=60.0):
    """
    Send the dump command, wait for a line beginning with "log_dump:",
    parse the hex data that follows, and return it as bytes.
    Any other lines (ESP_LOGx… etc.) are printed and skipped.
    """
    # fire off the dump command
    ser.write(b"log_dump\n")
    deadline = time.time() + timeout_s

    audit_logs = []
    while time.time() < deadline:
        raw = ser.readline()
        if not raw:
            # no data yet
            continue

        line = raw.decode(errors="ignore").strip()
        # is this our dump line?
        if line.startswith("log_dump:"):
            # extract the hex chunk after the colon
            hexstr = line.split(":", 1)[1]
            try:
                data = bytes.fromhex(hexstr)
                audit_logs.append(data)
            except ValueError:
                raise ValueError(f"Invalid hex in dump: {hexstr!r}")

        if line.startswith("log_dump_end:"):
            print(f"footer: received {len(audit_logs)} logs.")
            return audit_logs
            

        # otherwise it's just a normal log message—print it and keep waiting
        print("  log >", line)

    print(f"timeout: received {len(audit_logs)} logs.")
    return audit_logs


In [4]:
def verify_logs(audit_logs):
    """
    structure log: 
        - first 12 bytes = nonce
        - next 4 bytes = ad (timestamp)
        - next x bytes = ciphertext
        - last 16 bytes = prev_tag
    """

    aesccm = AESCCM(key, tag_length=16)
    results = []
    expected_nonce = None

    for idx, record in enumerate(audit_logs):
        if len(record) < 12 + 4 + 16:
            raise ValueError(f"Record {idx} too short to parse")

        nonce = record[:12]
        aad = record[12:16]
        ciphertext = record[16:-16]
        tag = record[-16]

        if expected_nonce is not None and noce != expected_nonce: 
            raise ValueError(f"Chain broken at record {idx}: nonce != previous tag")

        plaintext = aesccm.decrpyt(nonce, ciphertext + tag, aad)
        timestamp = int.from_bytes(aad, byteorder="little")

        results.append((timestamp, plaintext))
        expected_nonce = tag

    return results

In [5]:
send_ping_and_wait()

  log > I (13663) main: received command. cmd=ping
OK: ping:pong


In [6]:
audit_logs = send_log_dump_and_wait()

  log > D (15163) log_secure: logging secure. ts=15163
  log > D (15163) ringbuf_flash: ringbuf_flash_write writing log...
  log > D (15163) ringbuf_flash: WRITING hdr @ offset=28072 len=86 crc=0x1739961712
  log > D (15163) ringbuf_flash: Saving ringbuf metadata...
  log > D (15163) nvs: nvs_open_from_partition rb_log 1
  log > D (15163) nvs: nvs_set_blob log_secure 20
  log > D (15163) nvs: nvs_close 9
  log > D (15163) nvs: nvs_open_from_partition log_secure_nvs 1
  log > D (15163) nvs: nvs_set_blob meta 44
  log > D (15163) nvs: nvs_close 10
  log > D (20163) log_secure: logging secure. ts=20163
  log > D (20163) ringbuf_flash: ringbuf_flash_write writing log...
  log > D (20163) ringbuf_flash: WRITING hdr @ offset=28170 len=86 crc=0x3379454632
  log > D (20163) ringbuf_flash: Saving ringbuf metadata...
  log > D (20163) nvs: nvs_open_from_partition rb_log 1
  log > D (20163) nvs: nvs_set_blob log_secure 20
  log > D (20163) nvs: nvs_close 11
  log > D (20163) nvs: nvs_open_from_pa

[b"\x884@\x9c\x03\xefA\t\xc5\xf4\x08\xd8\xfbu\x00\x00\xdf\x96mS\xafvq1m&%O\xc8\x81X\xb7\xe8\x80\xba'h\xb4G\x90M\xf7\xc2\xc6\xd0z\xea-\x84I\x9bg\xe2t\xe4\x98f\x8d\xc5h4\xdd\x97\xd3\xf3X\\U\xe9\xfa\xa7\xe2HC4Sv\x9c\x06\xd5_6\x8a\x12\xd2f",
 b'\xa7\xe2HC4Sv\x9c\x06\xd5_6\x83\x89\x00\x002\xaf\x96\x16\x80\xedC\xf3x<*\x87\x04,-\xc4N6\xc0\x000\x19\xba\xa2ov\xf1\xd9P\x8f\xfe\xd9\xa9\xd3\x10\xf3!\xa6\xe9f\xadx\x8b\x88\xfa\x86\xa9\x1d\xa1k\x7f\xbb\xa8=\x9f\xc6\xa6Q\xb7T\xb9\xbe\x86H\xa0\x01s\xae\x0e\x05',
 b'\x9f\xc6\xa6Q\xb7T\xb9\xbe\x86H\xa0\x01\x0b\x9d\x00\x00\xe2\xabI\xc8\x8bvL\x88SLw\xd4\x0b\x1642\xbd\x085\x7f\x91-\xc0\xc1\xdc\xba\xaf\x9b\xe6f-\r\xafq\x93\xd3\x00\xda\xdfs\x85\x03\x92z\x1a\xf4\xa5\xd8\xc9X\xca~P\xca\xdfo0\x98\x9f\x8f\xf8{1\xb3\rD\x19\xf2q\xc6',
 b'\xdfo0\x98\x9f\x8f\xf8{1\xb3\rD\x93\xb0\x00\x00\x06\x88(\x96\xc1\xa5\xe2n\xa5\xea\x85xm_\xc6{}m-\xaf\xeb\r\n5\x0e$8r>\tG\x9c\xc1\xde\xeb\xf7\xbd\xec\x18\x97\xc4Vp\xd1\xab\xfc\xf2\x19\xeb.\x06vN!<\xbcN\xbf\xd1c<{\xf2\xa1y\xcd(>\xfd1

In [1]:
audit_logs = [b"\x884@\x9c\x03\xefA\t\xc5\xf4\x08\xd8\xfbu\x00\x00\xdf\x96mS\xafvq1m&%O\xc8\x81X\xb7\xe8\x80\xba'h\xb4G\x90M\xf7\xc2\xc6\xd0z\xea-\x84I\x9bg\xe2t\xe4\x98f\x8d\xc5h4\xdd\x97\xd3\xf3X\\U\xe9\xfa\xa7\xe2HC4Sv\x9c\x06\xd5_6\x8a\x12\xd2f",
 b'\xa7\xe2HC4Sv\x9c\x06\xd5_6\x83\x89\x00\x002\xaf\x96\x16\x80\xedC\xf3x<*\x87\x04,-\xc4N6\xc0\x000\x19\xba\xa2ov\xf1\xd9P\x8f\xfe\xd9\xa9\xd3\x10\xf3!\xa6\xe9f\xadx\x8b\x88\xfa\x86\xa9\x1d\xa1k\x7f\xbb\xa8=\x9f\xc6\xa6Q\xb7T\xb9\xbe\x86H\xa0\x01s\xae\x0e\x05',
 b'\x9f\xc6\xa6Q\xb7T\xb9\xbe\x86H\xa0\x01\x0b\x9d\x00\x00\xe2\xabI\xc8\x8bvL\x88SLw\xd4\x0b\x1642\xbd\x085\x7f\x91-\xc0\xc1\xdc\xba\xaf\x9b\xe6f-\r\xafq\x93\xd3\x00\xda\xdfs\x85\x03\x92z\x1a\xf4\xa5\xd8\xc9X\xca~P\xca\xdfo0\x98\x9f\x8f\xf8{1\xb3\rD\x19\xf2q\xc6',
 b'\xdfo0\x98\x9f\x8f\xf8{1\xb3\rD\x93\xb0\x00\x00\x06\x88(\x96\xc1\xa5\xe2n\xa5\xea\x85xm_\xc6{}m-\xaf\xeb\r\n5\x0e$8r>\tG\x9c\xc1\xde\xeb\xf7\xbd\xec\x18\x97\xc4Vp\xd1\xab\xfc\xf2\x19\xeb.\x06vN!<\xbcN\xbf\xd1c<{\xf2\xa1y\xcd(>\xfd1',
 b"<\xbcN\xbf\xd1c<{\xf2\xa1y\xcd\x1b\xc4\x00\x00\xdd\x9b\x80^E\x19' \xd2\xefG\x1dk\xfdiw}\xe3\xf3\xf9\xa6\xf7\xa1\xc7y\x98w\xe5\xe6\xacHH\\\x9a\x01\xff\xf5=\x13o~Bh*\xc6\x00\x92\xc4\n\xdd\xadS'(\x08Y%V\xd5\x97\x18MF\xc6\r\x9f\xaa\xb6Ed!",
 b"Y%V\xd5\x97\x18MF\xc6\r\x9f\xaa\xa3\xd7\x00\x00\x84$\x83'\xea\xcf\x85\xaf\xd4x\\\xeb\xa0\x8a\x01\xec\tD:\x95\xbc\xe1r;\x05\xbfxz\x9e\x81\xf0[\x9e\xc0\xb6\x00Mf\xf7\xd2\xee\x15 6\xc31\x1f\x0cY\xbe\x95\xd5\x88s\x84\xc7\x03K\xd8\x04\xf7\xa9\xa0\\tp\xdd\x8c4\xe2\xb1",
 b'\xc7\x03K\xd8\x04\xf7\xa9\xa0\\tp\xdd]\xeb\x00\x00\xcd\xb4\xf9Ppv5\x02\x90g\xf5\xef<\x9b\\\x9ef0\t\x08i\xab0\xc2\xcb\x0c/l\xd3\xb95\xfc\x96\xc7|os\xe8\xcf\xef\xec\xb6\x12`\xe5Wd\x98>(\x03\xa5\xc1\xec\xf4\xb8f\x15\xfa\x89\xf5\xb3\xd6\x9a(l\xe6\xbb\xe5\x81\x1a',
 b"\xb8f\x15\xfa\x89\xf5\xb3\xd6\x9a(l\xe6\xe5\xfe\x00\x00e\xd4\xea\xc7\xdd\xc2j\x91p\xb6}\xd4tmF\xe0\xc0\x89Gb]\xbb\x04\xd29\xb5\xe8o\xce\xcf\x0cY\xd7\x14J\xcc \x13\xf7\xda\x9a\x84\x8a\x082\xeaY\xa7\xd1\x8e|\xd6\x0e\xd6'y\x99+\xc9b\x92\xd0gCO0\x93\x9e\xd7\x0bw",
 b'y\x99+\xc9b\x92\xd0gCO0\x93m\x12\x01\x00\xef\xf0\r\x0f\x0c\xf0\x05\x06n\xaf\xe4\x84n\xfd\xaf2\xadb\x0e\xaa\xed\rz\x91\xe4LF\x1f\xe1\xb9\xb5\x18[\x13\x18"\x0f\xab&\x8a\x8eJC\x0c$\x01|\xf3\xae\x95w\x8bv\x97c\xfa\xccM\x85\xcc\xf7\xa4\xee\xca,S\xbf\xa3\x1a\xd4_',
 b'\xfa\xccM\x85\xcc\xf7\xa4\xee\xca,S\xbf\xf5%\x01\x00YH\xc4\x9b\xde\xd2)t\x0e\xf8\xbd\xe1\xea\r\t\x97\xd6\xba\xa2{m\x7f\xd6T\xec\xbd\xcd\xc0\x19\x15a\xda\xce\xe13\xa56_\xbf\x91\xc6{\xea\x05\x8b\xdb\xf6\xe3H,\xe2\x91\xa4\x1f\x9a\x82\x8c\x12\xfd\x8bB\x81X%\xa0\xa03\xa2K\xea\n',
 b'\x82\x8c\x12\xfd\x8bB\x81X%\xa0\xa03}9\x01\x00\x81$\xbcNE\xa0H\xebc/cR\x99\n@\xc3\xd4K\x86\x815\\)\xb7b\x85c1\xf6\r\t#T{#\x87\xa3}\xcf\x98\xdc\xc7\x01\x87\x92\xaa\xf8\x87\xd4\xb88\x00\xbc\x04\xfd\xc8P\xd5\xb7\xfc\x98\xc2G\x1f3\x9eS\x7f"Y\x92',
 b'\xc8P\xd5\xb7\xfc\x98\xc2G\x1f3\x9eS\x05M\x01\x00\x1f\xe5p\x95\xfbC\x87t\xf8x\xd9\xff\x0ea,\x0f9W\xe7\xfb[\x1a\x1d\xb7?\xb4\xc46~wQ\xe9\xad="\x14\x80\xc9\xee\xfa\x85C-\x14\xb3q\xc1\xdc\x8a\xc6\x86_3V\x00\xa7\x872Sg\x0b\x07y\x8d[O\xcc\xb6\xd7j\xc1',
 b'\xa7\x872Sg\x0b\x07y\x8d[O\xcc\x8d`\x01\x00\x1b\xa6\xadey\xcb~T\x03\xc2D\xdaVRi\xba\xc6\x8c\xd7S\xb90\xc3\x16<&P2\x0bZ\xfe\xa1bU\xc6*E\x0f\x91z\x80k\xbe\xaa\xe3Cb\r}\xae\x0c\xb4\xcb9\xd6*h\xef\x1d\xa3\xe5\rQ\x1f\xa4\x85\xfe-\xcamj',
 b'*h\xef\x1d\xa3\xe5\rQ\x1f\xa4\x85\xfe\x15t\x01\x00Mf\xff\x82\x9c\xe0\xaaM)J\xdc$\xaa*|\xdc\xe4o;\xfapQ\x1a\xadzX\xaa\xb5_D\x0c\x9a>\x1c\x98\xf6{\xba-\xaf\x1aw\xf7\xb7ubr\x95\xf4>d \x9b\xc5\xf3\xbbl\xf9\xe0:%+z\xc9\x08\xbdea\x81Le',
 b'\xbbl\xf9\xe0:%+z\xc9\x08\xbde\x9d\x87\x01\x002\xbf\x0e,\x01\xee\xe9}\x97\xfa\x9dc\xf0;\xba;@#\x8f\xb6\xa8,\x1f@\xf90O\xcd\xe5\xa3\x03\x92\xe3\xbf=\xd2\x13E\xd2[\xc8\x91\x8b\xfa\x99uu\xcb\xf5\x06\xcde\x00\xf5\xd9\x04L*\xddY\x8b\x0c/Uf$\xc8*\xc5\x16\xd4',
 b'\x04L*\xddY\x8b\x0c/Uf$\xc8%\x9b\x01\x00 \x1d\xb7VT\xf9\xf2\x0f\xa1+\xaf\x0c\x18\xd6s\xb5\xf6\x0b\x9b\x85\xaa\xb6\xfd\xea#.,\xde\x99}\xabQwJ\xce\xf2S\x05\xc9\x92\x16\x94\xc0.\xcc_|\xf1\x8c%q\x1bV\xd7\xa6\x02\xad5|\x86,\xaa\xca\x0c\\\x8aVA\xb2{\xaa',
 b"\x02\xad5|\x86,\xaa\xca\x0c\\\x8aV\xad\xae\x01\x00\xfaG\xac\xbf\xcaS\x194\x01S\xb9w\xd9q\xc5\x12\x85\xc2\x8b`\\\x0e\xcf^\xd8\xf7\xe3\xb0Vao\x9b_K, ZG\xff\xfe\xcc\xab#Y\x1d^\x10\xfd\x9f\xd5F@^\xd1]\xa4\xf0'\x99ah\xf3\x05\x83-\xe1\x8f\xb6\x08L ",
 b'\xa4\xf0\'\x99ah\xf3\x05\x83-\xe1\x8f5\xc2\x01\x00l\x13\xf4\x19\xd8\x98\xa8\xd4m\x80=\\81\xbb\x03\xc0`\xa8\xd3=\xc2*:Q\x1f\xcbk\xa6\xe5*\x16\xc9m"\x8a\x815\x85\xb7\xd1\x16\x0b\xcb\xe6\x8a\x16\xede\xaf\xbb\x18\x13@\x8b\xec+\xb6X\x8a\x9aL?\x8d\x1b\xa5\xed\x98\xdc\xd3\x83',
 b'\xec+\xb6X\x8a\x9aL?\x8d\x1b\xa5\xed\x8f\x00\x00\x00h=B\xf0\xc3\xf6>\x02\xb8l@\x8cs;\xa9ah\xa2\xbe\xc7\xe9\\\xa9R\xab}\xf9\xa5\x9f\x8f\x1f\xda\xf3C\xd8\x9a`\xd1^\xdc\xbe2:Xb\xfc\xc2\x9a\xe4\x0bw\x02?\xf9{\xe5\xbf/\x0bwQ\x08\xf9\xf7)o\xc8Rf\xe2',
 b'{\xe5\xbf/\x0bwQ\x08\xf9\xf7)o+\x14\x00\x007\x0f\xf0@\xa6\xf7\x12\x84U\x9a\x0bVIx\x91y\xb07\x08\xe07r\x9a\xa5\xea\x8e\x88\xdc\xb2k\x8f\x00\xdb\xcf\xact{\x95\xc0\xd5\xa6\xe47\xa0\xa2\xd1\xcd\x17\x06\xb9\x9f\xb0\xe5\\\xcc\xe70\xb2\x8d1\x81&\xeaJ\xea\x88\xc4F\xd0@',
 b"\xcc\xe70\xb2\x8d1\x81&\xeaJ\xea\x88\xb3'\x00\x00\xa7?g\x14\xf4\x1eR\x9d\xac1\x9ae?Ss$S\x1fa\xc5z\xca\xf6\x04\xd3\xa8\xe7\x05\xd3Z\x86l\xb3\xf7\xf9\x15\xf2\xec\x05;\xc2\xb9J\xcc<\xbc\xe0\x1a1\x9c*\x07\xefA\x19'\x87\x18F\x99\x16\r=\xf8\xed\xa8\xe8\xe0\x10\x81",
 b"\x19'\x87\x18F\x99\x16\r=\xf8\xed\xa8;;\x00\x00=\xd6NX\x8a\xdc\xccB\x94\xb9&H\x8d\xb94\x93\x98\xc2)\xaao\xb5:\xb9\x87A\xb05h\x05\xce\xc0\xfd\x9d\\\xa2\xbd+\xe1.l\x82v\xcc\xc6]\xea\x1dl\xde\xfcP\xd3Yv6\x05\x1e\xd7\x0b\x9e\xe3\xb5\xec\xe5\xcd6cK\x8e",
 b'v6\x05\x1e\xd7\x0b\x9e\xe3\xb5\xec\xe5\xcd\xc3N\x00\x00\x00eP\x95\x04Z\x07\xdc\xbdy\xdfM\xaa\x1a,\xa9\xe7\x0156R+\x1a\xb77Hl\xa5^\x10\x19\xc2E\xb9\xa2\xf8\x01\x1f\xe7\x88_\xe8kT\x03\x937S\xf0a\x16\xc4\xeeH\xcc\xd3\xcd\x06\x14\xca>\xaf\xe3\xe4\x85m\x01\xf3H\xe9']

In [ ]:
results = verify_logs(audi)